# Решение задачи "Классификация болезней яблонь"

В этом ноутбуке будем решать задачу классификации болезней яблонь с помощью предобученной нейронной сети ResNet-18.

In [1]:
# подключаем гугл диск на котором данные
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [2]:
!ls /content/gdrive/'My Drive'/datasets/2021_apple_clf

test  train


In [3]:
!ls /content/gdrive/'My Drive'/datasets/2021_apple_clf/train

0_гниль  1_ржавчина  2_парша


### Строим нейросеть

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models

In [7]:
PATH_TO_FOTO_TRAIN = '../content/gdrive/My Drive/datasets/2021_apple_clf/train'

Создаем даталоадеры для обучения нейросети:

In [8]:
images_dataset = torchvision.datasets.ImageFolder(PATH_TO_FOTO_TRAIN, transform=transforms.Compose([                                                                  
  transforms.ToTensor(),
  # нормализуем как в ImageNet
  torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                   [0.229, 0.224, 0.225]),
                                                                                                    
]))

images_dataloader = torch.utils.data.DataLoader(images_dataset, batch_size=16,
                                          shuffle=True)

Будем использовать предобученную на ImageNet сеть ResNet18:

In [9]:
net = models.resnet18(True, True).cuda() 
net  

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Заменим последний полносвязный слой сети на слой, который будет выдавать 3 значения на выходе (т.к. у нас 3 класса):

In [10]:
net.fc = nn.Linear(512, 3)

Заморозим все слом нейросети, кроме самого последнего, только что добавленного fc-слоя. Будем обучать только последний слой сети.

In [11]:
for i, child in enumerate(net.children()):
    if i == 9: 
        break
    for param in child.parameters():
        param.requires_grad = False  

Объявляем лосс-функцию и оптимизатор:

In [12]:
# стандартная лосс-функция для задачи классификации
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [13]:
# для обучения на GPU
device = 'cuda:0'
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Обучаем сеть 5 эпох:

In [14]:
n_epochs = 4
print_every = 10

total_step = len(images_dataloader)

for epoch in range(1, n_epochs+1):

    print(f'Epoch {epoch}\n')
    for batch_idx, (data, target) in enumerate(images_dataloader):
        # кладем данные на GPU
        data, target = data.to(device), target.to(device)

        # делаем шаг обучения сети
        optimizer.zero_grad()
        outputs = net(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

Epoch 1

Epoch [1/4], Step [0/25], Loss: 1.3772
Epoch [1/4], Step [20/25], Loss: 0.8002
Epoch 2

Epoch [2/4], Step [0/25], Loss: 0.6204
Epoch [2/4], Step [20/25], Loss: 0.3881
Epoch 3

Epoch [3/4], Step [0/25], Loss: 0.4582
Epoch [3/4], Step [20/25], Loss: 0.2036
Epoch 4

Epoch [4/4], Step [0/25], Loss: 0.1721
Epoch [4/4], Step [20/25], Loss: 0.4843


### Тестируем обученную нейросеть на тестовом наборе картинок:

In [15]:
PATH_TO_FOTO_TEST = '../content/gdrive/My Drive/datasets/2021_apple_clf/test'

In [16]:
images_testset = torchvision.datasets.ImageFolder(PATH_TO_FOTO_TEST, transform=transforms.Compose([                                                                  
  transforms.ToTensor(),
  torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                   [0.229, 0.224, 0.225]),
                                                                                                    
]))

images_testloader = torch.utils.data.DataLoader(images_testset, batch_size=1,
                                          shuffle=False)

In [17]:
batch_loss = 0
total=0
correct=0

with torch.no_grad():
        net.eval()

        for data, target in (images_testloader):
            # кладем данные на GPU
            data, target = data.to(device), target.to(device)
            outputs = net(data)
            # считаем loss
            loss = criterion(outputs, target)
            batch_loss += loss.item()

            # считаем accuracy
            _, pred = torch.max(outputs, dim=1)
            correct += torch.sum(pred==target).item()
            total += target.size(0)

        print("Acc", 100 * correct/total)
        print("Loss", batch_loss/len(images_testloader))

Acc 32.22222222222222
Loss 2.2905667373962286


In [18]:
#SAVE MODEL IN ORDER TO USE IT IN MOBILE APP
torch.save (net, 'apple_clf01.h5')


In [19]:
from google.colab import files

files.download('apple_clf01.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>